In [1]:
import pandas as pd
import pymysql
from datetime import date
import requests
from concurrent.futures import ThreadPoolExecutor
from itertools import combinations

# 데이터베이스 연결 설정
def connect_to_db():
    return pymysql.connect(
        host='database-1.cbcw28i2we7h.us-east-2.rds.amazonaws.com',
        user='kj',
        password='1234',
        database='nahonlab',
        charset='utf8mb4'
    )

API_KEY = "YYBjxJqR6CexItNR8rwA836gQiaGpQdSeeXCsAh8"

def calculate_age(birthday):
    """생년월일로부터 만 나이 계산"""
    today = date.today()
    age = today.year - birthday.year
    if (today.month, today.day) < (birthday.month, birthday.day):
        age -= 1
    return age

def recommend_exercises_from_db(user, tdee, exercise_list):
    """운동 추천 및 계획 생성"""
    # user 객체가 pandas.Series일 경우 딕셔너리로 변환
    if isinstance(user, pd.Series):
        user = user.to_dict()

    calories_to_burn = max(0, tdee - user.get('calories_intake', 2000))
    print(f"운동으로 소모해야 할 칼로리: {calories_to_burn:.2f} kcal")

    exercise_plan = []
    remaining_calories = calories_to_burn

    for _, exercise in exercise_list.iterrows():
        if remaining_calories <= 0:
            break

        max_reps = remaining_calories / exercise['calorie_per_cnt']
        reps = min(max_reps, 50)
        calories_burned = reps * exercise['calorie_per_cnt']
        exercise_plan.append({
            "exercise_name": exercise["exercise_name"],
            "category": exercise["category"],
            "reps": int(reps),
            "calories_burned": round(calories_burned, 2)
        })
        remaining_calories -= calories_burned

    print("\n추천 운동 계획:")
    for item in exercise_plan:
        print(f"- {item['exercise_name']} ({item['category']}): {item['reps']}회, {item['calories_burned']} kcal 소모")

    return exercise_plan

def fetch_exercise_list():
    connection = connect_to_db()

    try:
        with connection.cursor() as cursor:
            query = "SELECT exercise_name, category, calorie_per_cnt FROM exercise_list"
            cursor.execute(query)
            data = cursor.fetchall()
            columns = [col[0] for col in cursor.description]
            exercise_list = pd.DataFrame(data, columns=columns)
    finally:
        connection.close()

    return exercise_list

def determine_diet_strategy(user):
    """개인화된 식단 전략 결정"""
    strategies = []
    weight_diff = user['target_weight'] - user['current_weight']
    if weight_diff < -2:
        strategies.append('weight_loss')
    elif weight_diff > 2:
        strategies.append('muscle_gain')
    else:
        strategies.append('maintenance')

    age = calculate_age(user['birthday'])
    if age < 25:
        strategies.append('young_high_metabolism')
    elif age > 40:
        strategies.append('slower_metabolism')

    if user['sex'] == 'male':
        strategies.append('male_muscle_priority')
        if user['current_fat_percent'] > 20:
            strategies.append('fat_reduction')
    else:
        strategies.append('female_lean_priority')
        if user['current_fat_percent'] > 25:
            strategies.append('fat_reduction')

    if user['current_muscle_mass'] < user['target_muscle_mass']:
        strategies.append('muscle_build')

    return strategies

def calculate_macro_goals(user, diet_strategy):
    """개인화된 매크로 영양소 목표 설정"""
    base_calories = calculate_base_calories(user)
    if 'weight_loss' in diet_strategy:
        base_calories -= 300
    elif 'muscle_gain' in diet_strategy:
        base_calories += 300

    if 'muscle_gain' in diet_strategy:
        protein_ratio, carb_ratio, fat_ratio = 0.3, 0.4, 0.3
    elif 'weight_loss' in diet_strategy:
        protein_ratio, carb_ratio, fat_ratio = 0.35, 0.3, 0.35
    else:
        protein_ratio, carb_ratio, fat_ratio = 0.3, 0.4, 0.3

    base_protein = user['current_weight'] * (
        2.2 if 'muscle_gain' in diet_strategy else 
        1.6 if 'weight_loss' in diet_strategy else 
        1.8
    )

    base_carbs = (base_calories * carb_ratio) / 4
    base_fat = (base_calories * fat_ratio) / 9

    return base_carbs, base_protein, base_fat

def calculate_base_calories(user):
    """기초대사율(BMR) 계산"""
    current_weight = user['current_weight']
    current_height = user['current_height']
    age = calculate_age(user['birthday'])
    sex = user['sex']

    if sex == 'male':
        bmr = 88.362 + (13.397 * current_weight) + (4.799 * current_height) - (5.677 * age)
        activity_multiplier = 1.55
    else:
        bmr = 447.593 + (9.247 * current_weight) + (3.098 * current_height) - (4.330 * age)
        activity_multiplier = 1.45

    tdee = bmr * activity_multiplier
    return tdee

# 사용자 데이터 가져오기
try:
    connection = connect_to_db()
    with connection.cursor() as cursor:
        query = """
        SELECT 
            ui.user_id,
            ui.birthday,
            ui.sex,
            ubm.weight AS current_weight,
            ubm.height AS current_height,
            ubm.fat_percent AS current_fat_percent,
            ubm.muscle_mass AS current_muscle_mass,
            ug.target_weight,
            ug.target_fat_per,
            ug.target_muscle_mass
        FROM user_info ui
        JOIN user_body_metrics ubm ON ui.user_id = ubm.user_id
        JOIN user_goals ug ON ui.user_id = ug.user_id
        WHERE ubm.record_date = (
            SELECT MAX(record_date) 
            FROM user_body_metrics 
            WHERE user_id = ubm.user_id
        )
        """
        
        cursor.execute(query)
        metrics_data = cursor.fetchall()
        metrics_columns = [col[0] for col in cursor.description]
        metrics_df = pd.DataFrame(metrics_data, columns=metrics_columns)

finally:
    connection.close()

# 사용자 ID 입력받아 정보 출력
input_user_id = 12
user_data = metrics_df[metrics_df['user_id'] == input_user_id]

if not user_data.empty:
    user = user_data.iloc[0]
    diet_strategy = determine_diet_strategy(user)
    target_carbs, target_protein, target_fat = calculate_macro_goals(user, diet_strategy)

    target_carbs /= 3
    target_protein /= 3
    target_fat /= 3

    exercise_list = fetch_exercise_list()
    tdee = calculate_base_calories(user)
    exercise_plan = recommend_exercises_from_db(user, tdee, exercise_list)
else:
    print(f"User ID {input_user_id} not found.")


운동으로 소모해야 할 칼로리: 281.36 kcal

추천 운동 계획:
- Dips (Triceps): 50회, 75.0 kcal 소모
- Pullup (Back): 50회, 100.0 kcal 소모
- Pushup (Chest): 50회, 50.0 kcal 소모
- Squat (Legs): 37회, 56.36 kcal 소모


In [3]:
import pandas as pd
import pymysql
from datetime import date

# 데이터베이스 연결 설정
def connect_to_db():
    return pymysql.connect(
        host='database-1.cbcw28i2we7h.us-east-2.rds.amazonaws.com',
        user='kj',
        password='1234',
        database='nahonlab',
        charset='utf8mb4'
    )

def calculate_age(birthday):
    """생년월일로부터 만 나이 계산"""
    today = date.today()
    age = today.year - birthday.year
    if (today.month, today.day) < (birthday.month, birthday.day):
        age -= 1
    return age

def recommend_exercises_from_db(user, tdee, exercise_list):
    """운동 추천 및 계획 생성"""
    # user 객체가 pandas.Series일 경우 딕셔너리로 변환
    if isinstance(user, pd.Series):
        user = user.to_dict()

    calories_to_burn = max(0, tdee - user.get('calories_intake', 2000))
    print(f"운동으로 소모해야 할 칼로리: {calories_to_burn:.2f} kcal")

    exercise_plan = []
    remaining_calories = calories_to_burn

    for _, exercise in exercise_list.iterrows():
        if remaining_calories <= 0:
            break

        max_reps = remaining_calories / exercise['calorie_per_cnt']
        reps = min(max_reps, 50)
        calories_burned = reps * exercise['calorie_per_cnt']
        exercise_plan.append({
            "exercise_name": exercise["exercise_name"],
            "category": exercise["category"],
            "reps": int(reps),
            "calories_burned": round(calories_burned, 2)
        })
        remaining_calories -= calories_burned

    print("\n추천 운동 계획:")
    for item in exercise_plan:
        print(f"- {item['exercise_name']} ({item['category']}): {item['reps']}회, {item['calories_burned']} kcal 소모")

    return exercise_plan

def fetch_exercise_list():
    connection = connect_to_db()

    try:
        with connection.cursor() as cursor:
            query = "SELECT exercise_name, category, calorie_per_cnt FROM exercise_list"
            cursor.execute(query)
            data = cursor.fetchall()
            columns = [col[0] for col in cursor.description]
            exercise_list = pd.DataFrame(data, columns=columns)
    finally:
        connection.close()

    return exercise_list

def calculate_base_calories(user):
    """기초대사율(BMR) 계산"""
    current_weight = user['current_weight']
    current_height = user['current_height']
    age = calculate_age(user['birthday'])
    sex = user['sex']

    if sex == 'male':
        bmr = 88.362 + (13.397 * current_weight) + (4.799 * current_height) - (5.677 * age)
        activity_multiplier = 1.55
    else:
        bmr = 447.593 + (9.247 * current_weight) + (3.098 * current_height) - (4.330 * age)
        activity_multiplier = 1.45

    tdee = bmr * activity_multiplier
    return tdee

# 사용자 데이터 가져오기
try:
    connection = connect_to_db()
    with connection.cursor() as cursor:
        query = """
        SELECT 
            ui.user_id,
            ui.birthday,
            ui.sex,
            ubm.weight AS current_weight,
            ubm.height AS current_height,
            ubm.fat_percent AS current_fat_percent,
            ubm.muscle_mass AS current_muscle_mass,
            ug.target_weight,
            ug.target_fat_per,
            ug.target_muscle_mass
        FROM user_info ui
        JOIN user_body_metrics ubm ON ui.user_id = ubm.user_id
        JOIN user_goals ug ON ui.user_id = ug.user_id
        WHERE ubm.record_date = (
            SELECT MAX(record_date) 
            FROM user_body_metrics 
            WHERE user_id = ubm.user_id
        )
        """
        
        cursor.execute(query)
        metrics_data = cursor.fetchall()
        metrics_columns = [col[0] for col in cursor.description]
        metrics_df = pd.DataFrame(metrics_data, columns=metrics_columns)

finally:
    connection.close()

# 사용자 ID 입력받아 정보 출력
input_user_id = 15
user_data = metrics_df[metrics_df['user_id'] == input_user_id]

if not user_data.empty:
    user = user_data.iloc[0]
    exercise_list = fetch_exercise_list()
    tdee = calculate_base_calories(user)
    exercise_plan = recommend_exercises_from_db(user, tdee, exercise_list)
else:
    print(f"User ID {input_user_id} not found.")


운동으로 소모해야 할 칼로리: 216.86 kcal

추천 운동 계획:
- Dips (Triceps): 50회, 75.0 kcal 소모
- Pullup (Back): 50회, 100.0 kcal 소모
- Pushup (Chest): 41회, 41.86 kcal 소모
